In [138]:
import pandas as pd
import numpy as np
import collections
import gensim 
from gensim.models import word2vec, phrases
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation, strip_numeric,\
                    strip_non_alphanum, strip_multiple_whitespaces, strip_short
from textblob import TextBlob, Word

import re
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
%matplotlib inline


from strsimpy.cosine import Cosine

In [229]:
recipe = pd.read_csv('core-data_recipe.csv')
recipe = recipe.drop(columns=['image_url'])
recipe.head()

,recipe_id,recipe_name,ingredients,cooking_directions,nutritions
0,240488,"Pork Loin, Apples, and Sauerkraut",sauerkraut drained^Granny Smith apples sliced^...,{'directions': u'Prep\n15 m\nCook\n2 h 30 m\nR...,"{u'niacin': {u'hasCompleteData': False, u'name..."
1,218939,Foolproof Rosemary Chicken Wings,chicken wings^sprigs rosemary^head garlic^oliv...,"{'directions': u""Prep\n20 m\nCook\n40 m\nReady...","{u'niacin': {u'hasCompleteData': True, u'name'..."
2,87211,Chicken Pesto Paninis,focaccia bread quartered^prepared basil pesto^...,{'directions': u'Prep\n15 m\nCook\n5 m\nReady ...,"{u'niacin': {u'hasCompleteData': True, u'name'..."
3,245714,Potato Bacon Pizza,red potatoes^strips bacon^Sauce:^heavy whippin...,{'directions': u'Prep\n20 m\nCook\n45 m\nReady...,"{u'niacin': {u'hasCompleteData': True, u'name'..."
4,218545,Latin-Inspired Spicy Cream Chicken Stew,skinless boneless chicken breast halves^diced ...,{'directions': u'Prep\n10 m\nCook\n8 h 15 m\nR...,"{u'niacin': {u'hasCompleteData': False, u'name..."


In [230]:
df = recipe[['recipe_id','recipe_name', 'ingredients']]
df.replace("[^a-zA-Z]"," ",regex=True, inplace=True)

In [231]:
df.head()

,recipe_id,recipe_name,ingredients
0,240488,Pork Loin Apples and Sauerkraut,sauerkraut drained Granny Smith apples sliced ...
1,218939,Foolproof Rosemary Chicken Wings,chicken wings sprigs rosemary head garlic oliv...
2,87211,Chicken Pesto Paninis,focaccia bread quartered prepared basil pesto ...
3,245714,Potato Bacon Pizza,red potatoes strips bacon Sauce heavy whippin...
4,218545,Latin Inspired Spicy Cream Chicken Stew,skinless boneless chicken breast halves diced ...


In [232]:
df['details'] = df['recipe_name'].str.lower() + ', '+ df['ingredients'].str.lower()

C:\Users\wen\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [233]:
df.head()

,recipe_id,recipe_name,ingredients,details
0,240488,Pork Loin Apples and Sauerkraut,sauerkraut drained Granny Smith apples sliced ...,"pork loin apples and sauerkraut, sauerkraut ..."
1,218939,Foolproof Rosemary Chicken Wings,chicken wings sprigs rosemary head garlic oliv...,"foolproof rosemary chicken wings, chicken wing..."
2,87211,Chicken Pesto Paninis,focaccia bread quartered prepared basil pesto ...,"chicken pesto paninis, focaccia bread quartere..."
3,245714,Potato Bacon Pizza,red potatoes strips bacon Sauce heavy whippin...,"potato bacon pizza, red potatoes strips bacon ..."
4,218545,Latin Inspired Spicy Cream Chicken Stew,skinless boneless chicken breast halves diced ...,"latin inspired spicy cream chicken stew, skinl..."


In [234]:
details=[]
for row in range(0,len(df.index)):
    details.append(df.iloc[row,3])

In [235]:
details

['pork loin  apples  and sauerkraut, sauerkraut drained granny smith apples sliced large onion caraway seeds apple cider divided brown sugar rub  thai seasoning salt garlic powder ground black pepper boneless pork loin roast',
 'foolproof rosemary chicken wings, chicken wings sprigs rosemary head garlic olive oil lemon pepper seasoned salt',
 'chicken pesto paninis, focaccia bread quartered prepared basil pesto diced cooked chicken diced green bell pepper diced red onion shredded monterey jack cheese',
 'potato bacon pizza, red potatoes strips bacon sauce  heavy whipping cream butter minced garlic grated parmesan cheese crust  warm water      degrees f    degrees c  honey active dry yeast vegetable oil all purpose flour shredded mozzarella cheese',
 'latin inspired spicy cream chicken stew, skinless boneless chicken breast halves diced tomatoes green salsa black beans rinsed and drained pinto beans drained and rinsed kernel corn taco seasoning chopped fresh cilantro ground red chile pe

In [236]:
# input json  
#{
#    "uuid": 1234,
#    "user_id": 1234,
#    "request_id": 1,
#    "request_time": "9/28/2020",
#    "user input": {
#        "motherhood_stage": "pregnancy first trimester",
#        "ingredient_list": ["chicken", "tomato"],
#        "nutrition_focus": ["full-eval","iron"],
#        "if_vegetierian": true,
#        "ingredient_exclusion": ["onion","cilantro","cheese"]
#    }
#}

In [237]:
json_input = 'chicken tomato potato'

In [238]:
df['input'] = json_input

In [239]:
df.head()

,recipe_id,recipe_name,ingredients,details,input
0,240488,Pork Loin Apples and Sauerkraut,sauerkraut drained Granny Smith apples sliced ...,"pork loin apples and sauerkraut, sauerkraut ...",chicken tomato potato
1,218939,Foolproof Rosemary Chicken Wings,chicken wings sprigs rosemary head garlic oliv...,"foolproof rosemary chicken wings, chicken wing...",chicken tomato potato
2,87211,Chicken Pesto Paninis,focaccia bread quartered prepared basil pesto ...,"chicken pesto paninis, focaccia bread quartere...",chicken tomato potato
3,245714,Potato Bacon Pizza,red potatoes strips bacon Sauce heavy whippin...,"potato bacon pizza, red potatoes strips bacon ...",chicken tomato potato
4,218545,Latin Inspired Spicy Cream Chicken Stew,skinless boneless chicken breast halves diced ...,"latin inspired spicy cream chicken stew, skinl...",chicken tomato potato


In [240]:
cosine = Cosine(2)
df["p0"] = df["input"].apply(lambda s: cosine.get_profile(s)) 
df["p1"] = df["details"].apply(lambda s: cosine.get_profile(s)) 
df["cosine_sim"] = [cosine.similarity_profiles(p0,p1) for p0,p1 in zip(df["p0"],df["p1"])]

df_2 = df.drop(["p0", "p1"], axis=1)

In [253]:
searchfor = ['chicken','tomato','potato']
df_3 = df_2[df_2['details'].str.contains('|'.join(searchfor))]
df_4 = df_3[df_3['details'].str.contains('chicken')]

In [254]:
df_4

,recipe_id,recipe_name,ingredients,details,input,cosine_sim
1,218939,Foolproof Rosemary Chicken Wings,chicken wings sprigs rosemary head garlic oliv...,"foolproof rosemary chicken wings, chicken wing...",chicken tomato potato,0.248371
2,87211,Chicken Pesto Paninis,focaccia bread quartered prepared basil pesto ...,"chicken pesto paninis, focaccia bread quartere...",chicken tomato potato,0.297775
4,218545,Latin Inspired Spicy Cream Chicken Stew,skinless boneless chicken breast halves diced ...,"latin inspired spicy cream chicken stew, skinl...",chicken tomato potato,0.285421
8,258163,Schnitzel Sandwich,skinless boneless chicken breasts salt and gro...,"schnitzel sandwich, skinless boneless chicken ...",chicken tomato potato,0.251987
10,244786,Zucchini Noodles and Summer Vegetables with Sw...,zucchini cut into noodle shape strands extra v...,zucchini noodles and summer vegetables with sw...,chicken tomato potato,0.302058
11,260453,Fresh Balsamic Chicken Salad,balsamic vinegar English cucumbers thinly slic...,"fresh balsamic chicken salad, balsamic vinegar...",chicken tomato potato,0.397859
22,235757,Chicken Pot Pie a la Jeremy and Jasmin,butter chopped onion all purpose flour salt gr...,"chicken pot pie a la jeremy and jasmin, butter...",chicken tomato potato,0.257920
23,8778,Cajun Chicken Pasta,linguine pasta skinless boneless chicken breas...,"cajun chicken pasta, linguine pasta skinless b...",chicken tomato potato,0.253125
24,217969,Buffalo Chicken Pasta Salad,uncooked rotini pasta mayonnaise chunky blue c...,"buffalo chicken pasta salad, uncooked rotini p...",chicken tomato potato,0.280161
26,231459,Slow Cooker Pork with Mushrooms and Barley,garlic salt pork chops barley white mushrooms ...,"slow cooker pork with mushrooms and barley, ga...",chicken tomato potato,0.286034


In [255]:
df_3.shape

(15235, 6)

In [256]:
df_4.shape

(7410, 6)

In [265]:
output = df_4.nlargest(10, 'cosine_sim')

In [266]:
output

,recipe_id,recipe_name,ingredients,details,input,cosine_sim
3874,8905,Golden Vegetable Chicken,to pound whole chicken onion potatoes carro...,"golden vegetable chicken, to pound whole ch...",chicken tomato potato,0.643576
2617,241808,Chicken Stew,water chicken tenders carrots celery potatoes ...,"chicken stew , water chicken tenders carrots c...",chicken tomato potato,0.637307
32804,79383,Chunky Tomato Potato Soup,butter onions peeled cubed potatoes chopped ce...,"chunky tomato potato soup, butter onions peele...",chicken tomato potato,0.627022
43617,17900,Tomato Florentine Soup II,chicken stock tomato sauce tomato juice tomato...,"tomato florentine soup ii, chicken stock tomat...",chicken tomato potato,0.611132
494,16398,Chinese Chicken and Potato Soup,potatoes carrot turnip chopped onion garlic ch...,"chinese chicken and potato soup, potatoes carr...",chicken tomato potato,0.600751
44941,23996,Aaahh Potato Soup,Yukon Gold potatoes au gratin instant potato m...,"aaahh potato soup, yukon gold potatoes au gra...",chicken tomato potato,0.598353
9757,15737,Quick and Easy Enchiladas,corn oil onion tomato chicken chunks salt and ...,"quick and easy enchiladas, corn oil onion toma...",chicken tomato potato,0.591803
26141,23062,Quick Spaghetti Sauce,green bell pepper onion garlic tomatoes Italia...,"quick spaghetti sauce, green bell pepper onion...",chicken tomato potato,0.584048
25799,16965,Lower Fat Potato Soup,onion celery fat free chicken broth potatoes d...,"lower fat potato soup, onion celery fat free c...",chicken tomato potato,0.581954
32831,216719,Cabbage Potato and Tomato Soup,butter onion potatoes celery garlic water chic...,"cabbage potato and tomato soup, butter onion...",chicken tomato potato,0.579392


In [267]:
output.iloc[0,2]

'to   pound  whole chicken onion potatoes carrots sweet potato tomato mayonnaise cayenne pepper ground turmeric garlic powder salt and pepper to taste'

In [268]:
output.iloc[1,2]

'water chicken tenders carrots celery potatoes sweet potato peas tomato sauce bay leaves cooked rice'

In [269]:
output.iloc[2,2]

'butter onions peeled cubed potatoes chopped celery chopped carrots garlic Italian seasoning milk cornstarch tomatoes chicken broth tomato paste salt and pepper to taste'

In [270]:
output.iloc[3,2]

'chicken stock tomato sauce tomato juice tomato paste white sugar celery chopped carrots chopped spinach diced zucchini cooked pasta ground nutmeg salt and pepper to taste'

In [272]:
output.iloc[4,2]

'potatoes carrot turnip chopped onion garlic chicken leg salt and pepper to taste'

In [264]:
result = output[['recipe_id']]

In [186]:
result

,recipe_id
3874,8905
2617,241808
32804,79383
26119,20968
27879,14081
43617,17900
43144,21083
9001,15914
44941,23996
494,16398


In [276]:
nutrition = recipe[['recipe_id','nutritions']]

In [277]:
nutrition.head()

,recipe_id,nutritions
0,240488,"{u'niacin': {u'hasCompleteData': False, u'name..."
1,218939,"{u'niacin': {u'hasCompleteData': True, u'name'..."
2,87211,"{u'niacin': {u'hasCompleteData': True, u'name'..."
3,245714,"{u'niacin': {u'hasCompleteData': True, u'name'..."
4,218545,"{u'niacin': {u'hasCompleteData': False, u'name..."


In [279]:
from pandas.io.json import json_normalize
#nutritions_result = json_normalize(nutrition['nutritions'])

In [289]:
#import json

#with open('nutrition_2.txt', 'r') as fobj:
   # data = json.load(fobj)


In [290]:
arr = nutrition['nutritions'].to_numpy

In [292]:
#json_normalize(arr)